In [123]:
# libraries

import re

In [124]:
# config

#def get_meeting():
#    return 10

def get_vancouver():
    elements =  ["Author(s) of paper – family name and initials.", "Title of paper.", 
            "In: Editor(s) Family name and initials editor(s).","Title of conference.", 
            "Date of conference;", "Place of conference.", "Place of publication: Publisher’s name", 
            "Publication year", "p. Page numbers."]
#           Bengtsson S, Solheim BG. Enforcement of data protection in medical informatics. 
#           In: Lun KC, Degoulet P, Piemme TE, Reinhoff O, editors. MEDINFO 92. Proceedings of the 7th World Congress on Medical Informatics. 
#           1992 Sep 6‐10;          Geneva, Switzerland.    Amsterdam (NL): North Holland; 
#.          1992. p. 1561‐5.

def get_verbose():
    verbose = "no" # "yes"
    return verbose

In [125]:
# in

def read_raw(meeting):
    meeting    = str(meeting)
    eviprg_raw = open("{}th_eviprg_raw.txt".format(meeting)).read().split("\n")
    return eviprg_raw

In [132]:
# clean

def concat_lower_case(corrected_qtop):
    concatenated = []
    for i in range(len(corrected_qtop)-1):
        j      = i+1
        first  = corrected_qtop[i]
        second = corrected_qtop[j]
        if first[0].isupper():
            if second[0].islower():
                first_and_second = first+" "+second
                concatenated.append(first_and_second)
            else:
                concatenated.append(first)
    return concatenated

def correct_Qnnn_to_pnnn(has_p):
    corrected = []
    for snippet in has_p:
        sn_number = snippet[0]
        sn_text   = snippet[1]
        if len(sn_text)>2:    # page numbers
            if sn_text[-4]=="Q":
                sn_text   = sn_text.replace("Q", " p. ")
            corrected.append(sn_text)
    return corrected

def put_names_first(concatenated_lower_case, these_have_pdot, meeting):
    #meeting                 = get_meeting()
    shorthand_for_book      = generate_book(meeting)
    n_concat_lower_case     = enumerate(concatenated_lower_case)
    iterable                = range(len(concatenated_lower_case)-1)
    names_first             = [concatenated_lower_case[i+1]+". "+shorthand_for_book+concatenated_lower_case[i]+". " for i in iterable if i in these_have_pdot] 
    return names_first

def sniff_chapter(snippet):
    number = snippet[0]
    text   = snippet[1]
    return text[:7] == "Chapter"

def sniff_page(snippet):
    number = snippet[0]
    text   = snippet[1]
    return re.search("p\. ", text)!=None

def sniff_preface(snippet):
    number = snippet[0]
    text   = snippet[1]
    return text[:7] == "Preface"

def find_extra(numbered_raw):
    are_extra = list()
    for snippet in numbered_raw:
        sn_number = snippet[0]
        sn_text   = snippet[1]
        #print(number, text)
        if sniff_chapter(snippet):
            are_extra.append(sn_number)
    verbose = get_verbose()
    if verbose == "yes":
        print("are chapters:\n", are_extra)
    for snippet in numbered_raw:
        sn_number = snippet[0]
        sn_text   = snippet[1]
        #print(number, text)
        if sniff_preface(snippet):
            are_extra.append(sn_number)
    verbose = get_verbose()
    if verbose == "yes":
        print("are chapters then prefaces:\n", are_extra)        
    return are_extra

def spacedots_to_pages(no_extra):
    has_QQ = []
    for snippet in no_extra:
        sn_number = snippet[0]
        sn_text   = snippet[1]
        sn_Page   = sn_text.replace(" .", "QQ")
        has_QQ.append(sn_Page)
    has_Q_ = []
    for snippet in has_QQ:
        to_Q = re.sub("Q+", "Q", snippet)
        has_Q_.append(to_Q)
    has_p = []
    for snippet in has_Q_:
        to_p = re.sub("Q ", " p. ", snippet)
        has_p.append(to_p)
    has_pages = list(enumerate(has_p))
    return has_pages

# insert

def generate_book(meeting):
    assert meeting in (7,8,9,10,11), "are you sure about the meeting number?!"
    if meeting == 7:
        book = "In: Violence in Clinical Psychiatry. Eds: Needham I, Nijman H, Palmstierna T, Almvik R, Oud N. Proceedings of the {}th European Congress on Violence in Clinical Psychiatry. 19–22 October 2011; Prague. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2011. ".format(meeting)
    elif meeting == 8:
        book = "In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the {}th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. ".format(meeting)
    elif meeting == 9:
        book = "In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Duxbury J. Proceedings of the {}th European Congress on Violence in Clinical Psychiatry. 22–24 October 2015; Copenhagen. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2015. ".format(meeting)
    elif meeting == 10:
        book = "In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the {}th European Congress on Violence in Clinical Psychiatry. 26–28 October 2017; Dublin. Oud Consultancy, Amsterdam. 2017. ".format(meeting)
    elif meeting == 11:
        book = "In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the {}th European Congress on Violence in Clinical Psychiatry. 23–26 October 2019; Oslo. Oud Consultancy, Amsterdam. 2019. ".format(meeting)
    else:
        book = "BAD BOOK CHECK REFERENCE"
    return book

In [135]:
def put_titles_first(names_before_titles, meeting):
    
    print("meeting:\t", meeting)
    if meeting   == 7:
        year = 2011
    elif meeting == 8:
        year = 2013
    elif meeting == 9:
        year = 2015
    elif meeting == 10:
        year = 2017
    elif meeting == 11:
        year = 2019
    
    print("what is year:", year)
    
    for entry in names_before_titles:
        
        where_is_year = entry.find(str(year))
        where_is_pdot = entry.find("p.")
        print("what is entry:\n", entry)
        print("what is year:\t", year)
        print("where is year:\t", where_is_year)
        print("where is pdot:\t", where_is_pdot)
        
    return 1 #titles_first

def test_put_titles_first():
    references = [
        [["Bob Bowen, Michael R. Privitera, MD. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Doing What We Know When We Know It: Translating Research into Practice p. 454.", 
        "Stéphane Guay, Richard Boyer, Juliette Jarvis, Natasha Dugal. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Evaluation of changes in perceived self-efficacy in managing patient aggression after completing workplace prevention training p. 459.",
        "Adriana Mihai, Julian Beezhold, Loredana Buftea, Jon Snorranson, Cristian Damsa. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Training in management of aggressive behaviour - strengths and difficulties p. 462.", 
        "Jón Snorrason MSc., Guðmundur Sævar Sævarsson MSc., Hilmar Thor Bjarnason MSc.. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Experience of staff using manual restraint techniques p. 463.", 
        "Julian Beezhold, Lisa Vescio. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Reducing violence in Acute Inpatient Psychiatry p. 465.",
        "Cristian Damsa, Adriana Mihai. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. The training impact on management of agitation p. 467.",
        "Adriana Mihai, Lajos Domokos, Raluca Tirintica, Alina Radu, Aurel Nirestean. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Training in management of aggressive behaviour – experience in Romania p. 468.",
        "Jan Boogaarts & Bas van Raaij. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Interaction Skills Training (IST): Improving the efficacy of interaction and cooperation under non-cooperative circumstances p. 469.", 
        "Susan Vickory, Rhonda Tierney. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Unclear & Present Danger – Synthetic Stimulants & Synthetic Marijuana p. 470.", 
        "Elisabeth Myhre, Peter Jantzen. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Implemention of SOAS-R and BVC in a new youth forensic emergency Unit p. 471."], 8],
        [["Lizica Troneci & AnnMarie McDonald (USA). In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 10th European Congress on Violence in Clinical Psychiatry. 26–28 October 2017; Dublin. Oud Consultancy, Amsterdam. 2017. Reduction of patient assaults on the inpatient psychiatric units of an inner city hospital as a  p. uality Improvement Project using the PDSA Cycle p. 432. ",
        "Nienke de Bles, Elise van Wijk, Manoek Brobbel, Nathaly Rius-Ottenheim, Bert van. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 10th European Congress on Violence in Clinical Psychiatry. 26–28 October 2017; Dublin. Oud Consultancy, Amsterdam. 2017. Psychometrics of the Staff Observation Aggression Scale – Revised version (SOAS-R) in chronic psychiatric patients p. 434. "], 10],    
        [["Prof. Kevin S. Douglas (Canada). In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 11th European Congress on Violence in Clinical Psychiatry. 23–26 October 2019; Oslo. Oud Consultancy, Amsterdam. 2019. Violence risk assessment and management: What we know, and what we still need to focus on p. 31.",
        "Prof. Linda Gröning (Norway). In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 11th European Congress on Violence in Clinical Psychiatry. 23–26 October 2019; Oslo. Oud Consultancy, Amsterdam. 2019. Mental disorder and violent crime: Legal concepts, distinctions and problems p. 34.", 
        "Dr. Wilma Boevink (Netherlands). In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 11th European Congress on Violence in Clinical Psychiatry. 23–26 October 2019; Oslo. Oud Consultancy, Amsterdam. 2019. Towards a charter for collaboration between users and professionals p. 35.", 
        "Dr. Peter de Looff (Netherlands), Dr. Liza Cornet, Dr. Matthijs L. Noordzij, Dr. Mirjam. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Nijman H, Palmstierna T, Duxbury J. Proceedings of the 11th European Congress on Violence in Clinical Psychiatry. 23–26 October 2019; Oslo. Oud Consultancy, Amsterdam. 2019. Predicting aggression of patients and burnout symptoms in nursing staff with the use of biosensors p. 36."], 11]]
    
    for reference in references:
        names_before_titles = reference[0]
        year                = reference[1]
        titles_first        = put_titles_first(names_before_titles, meeting)
        print(i)
    
test_put_titles_first()

meeting:	 11
what is year: 2019
what is entry:
 Bob Bowen, Michael R. Privitera, MD. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Doing What We Know When We Know It: Translating Research into Practice p. 454.
what is year:	 2019
where is year:	 -1
where is pdot:	 388
what is entry:
 Stéphane Guay, Richard Boyer, Juliette Jarvis, Natasha Dugal. In: Violence in Clinical Psychiatry. Eds: Callaghan P, Oud N, Bjørngaard JH, Nijman H, Palmstierna T, Almvik R, et al. Proceedings of the 8th European Congress on Violence in Clinical Psychiatry. 23–26 October 2013; Ghent. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2013. Evaluation of changes in perceived self-efficacy in managing patient aggression after completing workplace prevention training p. 459.
what

In [128]:
# main

if __name__=="__main__":
    
    # single meeting
    #meeting                 = get_meeting()
    
    # or a series
    meetings                = [7,8,9,10,11]
    for meeting in meetings:
        eviprg_raw              = read_raw(meeting)
        numbered_raw            = list(enumerate(eviprg_raw))
        are_extra               = find_extra(numbered_raw)
        no_extra                = [sn for sn in numbered_raw if sn[0] not in are_extra]  
        has_pages               = spacedots_to_pages(no_extra)
        corrected_qtop          = correct_Qnnn_to_pnnn(has_pages)
        concatenated_lower_case = concat_lower_case(corrected_qtop)
        these_have_pdot         = [snippet[0] for snippet in enumerate(concatenated_lower_case) if sniff_page(snippet)]
        names_before_titles     = put_names_first(concatenated_lower_case, these_have_pdot, meeting)
        for i in names_before_titles:
            print(i)
        #titles_first            = put_titles_first()
    
    
    
        

Stephen Hart (Canada). In: Violence in Clinical Psychiatry. Eds: Needham I, Nijman H, Palmstierna T, Almvik R, Oud N. Proceedings of the 7th European Congress on Violence in Clinical Psychiatry. 19–22 October 2011; Prague. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2011. The Comprehensive Assessment of Psychopathic Personality (CAPP) p. 27. 
Caroline Logan Michael Doylee (UK). In: Violence in Clinical Psychiatry. Eds: Needham I, Nijman H, Palmstierna T, Almvik R, Oud N. Proceedings of the 7th European Congress on Violence in Clinical Psychiatry. 19–22 October 2011; Prague. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2011. The Short-Term Assessment of Risk and Treatability (START): measuring and managing risk in the short-term p. 29. 
Anton Došen (Netherlands). In: Violence in Clinical Psychiatry. Eds: Needham I, Nijman H, Palmstierna T, Almvik R, Oud N. Proceedings of the 7th European Congress on Violence in Clinical Psychiatry. 19–22 October 2011; Prague. Kavanah, Dwing

In [129]:
# tests

def test_concat_lower_case():
    corrected_qtop = ['Potential severity of aggressive behaviour after acquired brain injury:', 
                      'implications for recording p. 162',
     'Geoff Dickens, Nick Alderman, Len Bowers', 'St Andrew’s Academic Centre, Northampton, UK', 
     'Violence and mental disorder: A central paradigm for psychiatry p. 166', 'Virgil Hancock', 
     'Carondelet Hospitals, Tucson, Arizona, USA', 
     'Safety in numbers: is the violence research literature too heterogeneous to be useful? p. 172', 
     'Juliet Hockenhull, Richard Whittington, Maria Leitner, M Gemma Cherry, Wally Barr, James', 
     'McGuire, Rumona Dickson. University of Liverpool, Liverpool, UK', 
     'Partnering with an inpatient psychiatric hospital to evaluate measures of', 'aggressive behavior p. 173', 
     'Joanne Iennaco, Lawrence Scahill, Jane Dixon, Robin Whittemore, Susan Busch, Len Bowers', 
     'Yale University, New Haven, USA', 
     'I’m just doing fine: Preliminary results of an art and training project relating to', 
     'self-harm in psychiatry p. 174', 'Nienke Kool, Berno van Meijel, Bauke Koekkoek, Ad Kerkhof', 
     'Palier, Intensive Treatment Centre, The Hague, The Netherlands', 
     'Audit of suicides in a South Staffordshire (UK) psychiatry unit to determine any',
     'preventable deaths p. 176', 'Vasudevan Krishnan, Jayaraj Padmanabhan, Abid Khan', 
     'South Staffordshire and Shropshire NHS Trust, Stafford, UK', 
     'Introducing screening for domestic violence in family care settings in Slovenia p. 177',
     'Nena Kopcavar Gucek, Igor Svab, Polona Selic', 
     'Department of Family Medicine, Medical School, University of Ljubljana, Ljubljana, Slovenia', 
     'A comparative study of attitudes towards aggression in 16 psychiatric wards p. 178', 
     'Tero Laiho, Hanna Putkonen, Nina Lindberg', 
     'Helsinki University Central Hospital, Clinic of Acute Psychiatry, Peijas Hospital, Vantaa, Finland',
     'Available instruments to register aggression incidents on psychiatric wards: a', 
     'literature review p. 179', 
     'Joris Leys, Karen Lauwaert, Nataly Fillion, Micheline Gobert, Katrien Vanderwee, Sofie Verhaeghe',
     'Ghent University, Ghent, Belgium', 'Demographics on not formerly known inpatients in an medium security forensic', 
     'department in Denmark p. 180', 'Steen Madsen, Christian Delcomyn Steffensen, Thomas Schütze',
     'Psykiatrisk Center Sankt Hans, Roskilde, Denmark', 
     'Aggression in psychiatric settings: incident registration using SOAS-R in nine', 
     'Belgian hospitals p. 181', # annoying
     'Joris Leys, Karen Lauwaert, Nataly Fillion, Jasper Feyaerts, Micheline Gobert, Katrien', 
     'Vanderwee, Sofie Verhaeghe. Ghent University, Ghent, Belgium']
    concatenated = concat_lower_case(corrected_qtop)
    #for i in enumerate(concatenated):
    #    assert i[1].isupper(), "should be upper"
    #print("pass concat lower case")

def test_correct_Qnnn_to_pnnn():
    has_p = [(750, 'An exchange program for psychiatric nurses of acute wards to reduce the use of'),
    (751, 'seclusion p. 454'),
    (752, 'Remy Welleman, Elleke Landeweer, Cecile Gijsbers van Wijk, Guy Widdershoven'),
    (753, 'GGZ inGeest, Amsterdam, The Netherlands'),
    (754, 'Association between physical environmental factors and levels of conflict and'),
    (755, 'containment on psychiatric wards in the NetherlandsQ455'),
    (756, 'Petra van der Schaaf, Femmy Keuning,Elise Dusseldorp, Eric Noorthoorn, Wim Janssen'),
    (757, 'TNO, Dutch Centre for Health Assets/DuCHA, Utrecht, The Netherlands'),
    (758, 'Variation in coercive measures over a large Dutch sample: understanding'),
    (759, 'differences p. 457'),
    (760, 'Eric Noorthoorn, Wim Janssen, Adriaan Hoogendoorn, Petra van der Schaaf, Femmy Keunig,'),
    (761, 'Guy Widdershoven, Henk Nijman, The Netherlands'),
    (762, 'Staffing and patients characteristics as determinants of seclusionQ460'),
    (763, 'Wim Janssen, Petra van der Schaaf Eric Noorthoorn, Len Bowers, CL Mulder, Annet Smit,'),
    (764, 'Henk Nijman, Guy Widdershoven'),
    (765, 'GGNet Warnsveld, The Netherlands')]
    corrected = correct_Qnnn_to_pnnn(has_p)
    for i in corrected:
        assert i[-3] != "Q", "should not be Q"
    print("passed Q to p")
    
def test_put_names_first():
    meeting = 7
    these_have_pdot         = [0, 2, 6, 8, 11, 14, 17, 20, 23, 26, 28, 31, 34, 37, 40]
    concatenated_lower_case = ['The Comprehensive Assessment of Psychopathoc Personality (CAPP) p. 27', 'Stephen Hart (Canada)', 'The Short-Term Assessment of Risk and Treatability (START): measuring and managing risk in the short-term p. 29', 'Caroline Logan Michael Doylee (UK)', 'European Association in Mental Health and Intellectual Disability (EAMHID)', 'Guidelines for Assessment, Diagnostic and Treatment of Severe Problem', 'Behaviour in Persons with Intellectual Disability p. 30', 'Anton Došen (Netherlands)', 'The bulldozer and the ballet dancer: Aspects of nurses’ caring approaches in acute psychiatric intensive care p. 31', 'Anna Björkdahl, Tom Palmstierna, Hansebo Görel', 'SLSO, Health Care Provision, Stockholm County Council, Stockholm, Sweden', 'The possibilities and pitfalls in violence risk assessment and management p. 33', 'Patrick Callaghan', 'University of Nottingham, UK', 'Violence and Mandated Community Treatment: The MacArthur Studies p. 34', 'John Monahan', 'School of Law, University of Virginia, Charlottesville, USA', 'Be the agent of the change you want to see in the world: Reducing restrictive interventions in human services p. 39', 'Sharon Paley', 'British Institute of Learning Disabilities, Kidderminster, UK', 'Schizophrenia, treatment adherence, substance use, and violence p. 42', 'Jan Volavka', 'New York University School of Medicine, New York', 'Measuring and managing short-term risk: Making the START work in practice p. 50', 'Mike Doyle RMN, PhD & Caroline Logan DPhil, Greater Manchester West Mental Health', 'NHS Foundation Trust & University of Manchester (UK)', 'Markers and prediction of violent behavior in general population p. 57', 'Dr.Marek Susta, PhD', 'The effective care and control of patients with psychopathic Personality Disorder: challenges and solutions p. 58', 'Professor Stephen D. Hart, PhD, Department of Psychology, Simon Fraser University, Burnaby, BC', 'Canada']
    names_first             = put_names_first(concatenated_lower_case, these_have_pdot, meeting)
    assert names_first[0] == 'Stephen Hart (Canada). In: Violence in Clinical Psychiatry. Eds: Needham I, Nijman H, Palmstierna T, Almvik R, Oud N. Proceedings of the 7th European Congress on Violence in Clinical Psychiatry. 19–22 October 2011; Prague. Kavanah, Dwingeloo and Oud Consultancy, Amsterdam. 2011. The Comprehensive Assessment of Psychopathoc Personality (CAPP) p. 27. '
    print("passed put names first")
    # visual check
    
def test_read_raw():
    meeting = 7
    raw     = read_raw(meeting)
    assert type(raw)==list, "should be a list"
    assert type(raw[0])==str, "should be a string"
    print("passed read raw")
    
def test_sniff_page():
    concatenated_lower_case = [
    (79, 'German guardianships, involuntary admissions and physical restraint measures in psychiatry: Implications for community-based psychiatry? p. 99'),
    (80, 'Juan Valdes-Stauber, Reinhold Kilian, Horst Deinert'),
    (81, 'SINOVA Klinik Ravensburg, Ravensburg, Germany'),
    (82, 'A survey about knowledge and attitudes of people towards violence against women in community / family settings p. 103'),
    (83, 'Aziz Batebi, Abolghasem Pourreza, Pedram Mousavi, Gholamreza Garmaroudi'),
    (84, 'Dept of Health Education & Promotion, School of Public Health. Tehran University of'),
    (85, 'Medical Sciences, Tehran, Iran'),
    (86, 'Spousal violence against women in Iran p. 104'),
    (87, 'Hassan Eftekhar, Ameneh Setareh Forouzan'),
    (88, 'Health Promotion and Education Department/ School of Public Health/ Tehran University of'),
    (89, 'Medical Sciences, Tehran, Iran')]

    answer = [True, False, False, 
              True, False, False, False,
              True, False, False, False]
    
    smell_like_pages = [sniff_page(snippet) for snippet in concatenated_lower_case]
    assert smell_like_pages == answer, "should be able to find pages"
    
    
def test_spacedots_to_pages():

    no_extra = [(2, 'The Comprehensive Assessment of Psychopathic Personality (CAPP) . . . . . . . . . . . . . . . . . . . . . 27'),
    (3, 'Stephen Hart (Canada)'), 
    (4, 'The Short-Term Assessment of Risk and Treatability (START): measuring and'), 
    (5, 'managing risk in the short-term . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29'), 
    (6, 'Caroline Logan Michael Doylee (UK)'), 
    (7, 'European Association in Mental Health and Intellectual Disability (EAMHID)'), 
    (8, 'Guidelines for Assessment, Diagnostic and Treatment of Severe Problem'), 
    (9, 'Behaviour in Persons with Intellectual Disability . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 30'), 
    (10, 'Anton Došen (Netherlands)'), 
    (12, 'The bulldozer and the ballet dancer: Aspects of nurses’ caring approaches in'), 
    (13, 'acute psychiatric intensive care . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 31'), 
    (14, 'Anna Björkdahl, Tom Palmstierna, Hansebo Görel'), 
    (15, 'SLSO, Health Care Provision, Stockholm County Council, Stockholm, Sweden'), 
    (16, 'The possibilities and pitfalls in violence risk assessment and management . . . . . . . . . . . . . . . 33'), 
    (17, 'Patrick Callaghan'), 
    (18, 'University of Nottingham, UK'), 
    (19, 'Violence and Mandated Community Treatment: The MacArthur Studies . . . . . . . . . . . . . . . . . 34'), 
    (20, 'John Monahan'), 
    (21, 'School of Law, University of Virginia, Charlottesville, USA'), 
    (22, 'Be the agent of the change you want to see in the world: Reducing restrictive'), 
    (23, 'interventions in human services . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 39'),
    (24, 'Sharon Paley'), 
    (25, 'British Institute of Learning Disabilities, Kidderminster, UK')]
    has_pages = spacedots_to_pages(no_extra)
    #print("has pages:\t", has_pages)
    assert type(has_pages)==list, "wrong type"
    print("passed spacedots to pages")
    
def tests():
    test_concat_lower_case()
    test_correct_Qnnn_to_pnnn()
    test_put_names_first()
    test_read_raw()
    test_sniff_page()
    test_spacedots_to_pages()
    print("passed all tests")
tests()

passed Q to p
passed put names first
passed read raw
passed spacedots to pages
passed all tests
